In [1]:
import anndata as ad
import numpy as np
import scipy as sp
import pandas as pd
import seaborn as sns
import sys
import torch
import os
import matplotlib.pyplot as plt

from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import normalize
from sklearn.metrics import pairwise_distances

/usr/local/lib/python3.6/dist-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


### Neighborhood

In [2]:
meta = pd.read_csv('../cite.rna.meta.csv')
merge = pd.read_csv('../results/raw/Seurat_pca.csv') #,header=None)
out_x = merge.iloc[0:15066,1:].to_numpy()
out_y = merge.iloc[15066:,1:].to_numpy()
merge = merge.iloc[:,1:].to_numpy()


In [3]:
match_matrix = pairwise_distances(out_x, 
                                  out_y, metric="euclidean")
#match_matrix = np.exp(-match_matrix)

batch_code = meta['batch'].to_numpy()

match_matrix_1 = match_matrix[batch_code=='s4d1',:]
match_matrix_1 = match_matrix_1[:,batch_code=='s4d1']

match_matrix_2 = match_matrix[batch_code=='s4d8',:]
match_matrix_2 = match_matrix_2[:,batch_code=='s4d8']

match_matrix_3 = match_matrix[batch_code=='s4d9',:]
match_matrix_3 = match_matrix_3[:,batch_code=='s4d9']

In [4]:
neighbor_1 = np.argsort(match_matrix_1, axis=1, kind='stable')
neighbor_1 = np.argsort(neighbor_1, axis=1, kind='stable')

neighbor_1 = np.argsort(match_matrix_1, axis=1, kind='stable')
neighbor_1 = np.argsort(neighbor_1, axis=1, kind='stable')
writer = open('../results/raw/neighborhood_s4d1_Seurat.txt','w')
writer.write('\n'.join(np.diag(neighbor_1).astype('str').tolist()))
writer.close()


neighbor_2 = np.argsort(match_matrix_2, axis=1, kind='stable')
neighbor_2 = np.argsort(neighbor_2, axis=1, kind='stable')
writer = open('../results/raw/neighborhood_s4d8_Seurat.txt','w')
writer.write('\n'.join(np.diag(neighbor_2).astype('str').tolist()))
writer.close()


neighbor_3 = np.argsort(match_matrix_3, axis=1, kind='stable')
neighbor_3 = np.argsort(neighbor_3, axis=1, kind='stable')
writer = open('../results/raw/neighborhood_s4d9_Seurat.txt','w')
writer.write('\n'.join(np.diag(neighbor_3).astype('str').tolist()))
writer.close()

### Silhoutte score

In [5]:
meta = pd.read_csv('../cite.rna.meta.csv')
merge = pd.read_csv('../results/raw/Seurat_pca.csv') #,header=None)
out_x = merge.iloc[0:15066,1:].to_numpy()
out_y = merge.iloc[15066:,1:].to_numpy()
merge = merge.iloc[:,1:].to_numpy()

In [4]:
from sklearn.metrics import silhouette_score
merge = np.concatenate((out_x, out_y), axis=0)

ct_int = meta['cell_type'].astype('category').cat.codes.to_numpy().astype('int')

ct_int = np.concatenate((ct_int,ct_int), axis=0)
tech_int = np.concatenate((np.zeros(meta.shape[0]),
                          np.ones(meta.shape[0])), axis=0)
batch_int = np.concatenate((meta['batch'].to_numpy(),
                            meta['batch'].to_numpy()), axis=0)

silhouette_ct = silhouette_score(merge, ct_int)
silhouette_tech = silhouette_score(merge, tech_int)
silhouette_batch = silhouette_score(merge, batch_int)


In [5]:
print(silhouette_ct)
print(1-silhouette_tech)
print(1-silhouette_batch)


-0.03814076409514854
0.9448424733985693
1.0069581123900704


### Save embed

In [2]:
meta = pd.read_csv('../cite.rna.meta.csv')
merge = pd.read_csv('../results/raw/Seurat_pca.csv') #,header=None)
out_x = merge.iloc[0:15066,1:].to_numpy()
out_y = merge.iloc[15066:,1:].to_numpy()
#merge = merge.iloc[:,1:].to_numpy()


In [3]:
embed_df = pd.DataFrame(np.concatenate((out_y,out_x),axis=0))
embed_df.insert(0,'index',merge['Unnamed: 0'].to_numpy())
embed_df.to_csv('../results/raw/Seurat_embed.csv',index=None)